# Homework 2 (Due Sep. 27) 
Problems marked with a (\*) are only required for graduate students.  Undergrads may undertake them for extra credit worth half the problems point value, with no penalties incurred for an incorrect answer.

## 1. Bayesian Networks/Conditional Probability

Consider three binary variables $a, b, c \in \{0, 1\}$ having the joint distribution given by  

In [1]:
import numpy as np
#        | a | b | c | P(a,b,c)
P_table = (
np.array([[0,  0,  0,  0.192 ],
          [0,  0,  1,  0.144 ],
          [0,  1,  0,  0.048 ],
          [0,  1,  1,  0.216 ],
          [1,  0,  0,  0.192 ],
          [1,  0,  1,  0.064 ],
          [1,  1,  0,  0.048 ],
          [1,  1,  1,  0.096 ]]))

# Convert P_table (which is a probability table) to a probability array, which 
# is much more computationally sensible for a discrete valued distribution
P_abc = np.zeros((2,2,2))
for entry in P_table:
    a = int(entry[0])
    b = int(entry[1])
    c = int(entry[2])
    P = entry[3]
    P_abc[a,b,c] = P

Note that you may want to use the following functions to help you in the following problem:

In [2]:
def marginalize(P,variables_to_marginalize=(),keepdims=False):
    """ Marginalize a probability table, i.e. compute P(a,b) = sum_c P(a,b,c) 
    
        Arguments: 
        
        P (array) -> a probability array in which the dimensions correspond to random variables,
                     the indices to variable values, and the entries to probabilities
        variables_to_marginalize (tuple) -> a list of integers containing the 
                                    variable numbers to marginalize over
        keepdims (boolean) -> Marginalization reduces the dimensionality of the distribution.  keepdims=False
                       removes that dimension from the array indexing scheme.  For example P(a,b,c) has
                       an array that is 2x2x2.  If we marginalize over variable/index 1 (aka b), then the 
                       resulting array P(a,c) is 2x2 if keepdims=False (which may change the index of a 
                       given variable.  For example, after marginalizing, variable/index 1 is now c).  
                       Conversely, the array becomes 2x1x2 if keepdims=True.
    
    """
    # Sum over the axes given in variables_to_marginalize
    return np.sum(P,axis=variables_to_marginalize,keepdims=keepdims)

def condition(P,variables_to_condition=()):
    """ Condition a probability table, i.e. compute P(a|c) = P(a,c)/P(c) 
    
        Arguments: 
        
        P (array) -> a probability array in which the dimensions correspond to random variables,
                     the indices to variable values, and the entries to probabilities
        variables_to_condition (tuple) -> a list of integers containing the variables to condition on
        
        NOTE: This function always returns an array that is the same size as the input.  However, this array
        no longer sums to one as does an unconditioned joint distribution.  Instead, it sums to one only given
        a particular value of the conditioned variable/index.  For example:
        
        P_abc.sum()==1 -> True
        P_ab_given_c = condition(P_abc,variables_to_condition=(2,))
        P_ab_given_c.sum()==1 -> False
        P_ab_given_c[:,:,0].sum()==1 -> True
        P_ab_given_c[:,:,1].sum()==1 -> True
   
    """
    # find the variables to marginalize over to get the marginal distribuion of the
    # variables that we wish to condition on
    v_to_m = list(range(P.ndim))
    for v in variables_to_condition:
        v_to_m.remove(v)
        
    # Compute the conditional by dividing the input by the output
    return P/marginalize(P,variables_to_marginalize=tuple(v_to_m),keepdims=True)
    
P_ab = marginalize(P_abc,variables_to_marginalize=(2,))
P_a_given_b = condition(P_ab,variables_to_condition=(1,))


### Joint distribution equivalency (20 pts)
Show by direct evaluation that $P(a,b,c) = P(a)P(c|a)P(b|c)$ for all values of $a$,$b$, and $c$.

In [3]:
for i in range(2):
    for j in range(2):
        for k in range(2):
            P_all = P_abc[i,j,k]
            print("\nP(a=" + str(i) + ",b=" + str(j) + ",c=" + str(k) + ") = " + str(P_all))
            
            P_a = marginalize(P_abc,variables_to_marginalize=(1,2))
            print("\nP(a=" + str(i) + ") = " + str(P_a[i]))
            
            P_ac = marginalize(P_abc,variables_to_marginalize=(1,))
            P_c_given_a = condition(P_ac,variables_to_condition=(0,))
            print("P(c=" + str(k) + "|a=" + str(i) +") = " + str(P_c_given_a[k][i])) 

            P_bc = marginalize(P_abc,variables_to_marginalize=(0,))
            P_b_given_c = condition(P_bc,variables_to_condition=(1,))
            print("P(b=" + str(j) + "|c=" + str(k) +") = " + str(P_b_given_c[j][k])) 
            
            P_product = P_a[i] * P_c_given_a[k][i] * P_b_given_c[j][k]
            print("\nP(a=" + str(i) + ")P(c=" + str(k) + "|a=" + str(i) +")P(b=" + str(j) + "|c=" + str(k) +") = " + str(P_product))
            print("______________________________________")



P(a=0,b=0,c=0) = 0.192

P(a=0) = 0.6
P(c=0|a=0) = 0.4
P(b=0|c=0) = 0.8

P(a=0)P(c=0|a=0)P(b=0|c=0) = 0.192
______________________________________

P(a=0,b=0,c=1) = 0.144

P(a=0) = 0.6
P(c=1|a=0) = 0.6
P(b=0|c=1) = 0.39999999999999997

P(a=0)P(c=1|a=0)P(b=0|c=1) = 0.144
______________________________________

P(a=0,b=1,c=0) = 0.048

P(a=0) = 0.6
P(c=0|a=0) = 0.4
P(b=1|c=0) = 0.2

P(a=0)P(c=0|a=0)P(b=1|c=0) = 0.048
______________________________________

P(a=0,b=1,c=1) = 0.216

P(a=0) = 0.6
P(c=1|a=0) = 0.6
P(b=1|c=1) = 0.6

P(a=0)P(c=1|a=0)P(b=1|c=1) = 0.216
______________________________________

P(a=1,b=0,c=0) = 0.192

P(a=1) = 0.4
P(c=0|a=1) = 0.6
P(b=0|c=0) = 0.8

P(a=1)P(c=0|a=1)P(b=0|c=0) = 0.192
______________________________________

P(a=1,b=0,c=1) = 0.064

P(a=1) = 0.4
P(c=1|a=1) = 0.39999999999999997
P(b=0|c=1) = 0.39999999999999997

P(a=1)P(c=1|a=1)P(b=0|c=1) = 0.064
______________________________________

P(a=1,b=1,c=0) = 0.048

P(a=1) = 0.4
P(c=0|a=1) = 0.6
P(b=1|c=0) = 0.

### Graph Visualization (10 pts)

Visualize the corresponding directed graph, either using graph software like [networkx](https://networkx.github.io/) or drawing it by hand.

![title](graphs/Graph1.png)

### A Trick Question in Graphs (\*) (10 pts)
Plot the graph associated with
$$
P(A,B,C,D,E) = P(A)P(C)P(B|A,C)P(D|C)P(E|D),
$$
where $A,B,C,D,E \in \{0,1\}$ with conditional probability tables
\begin{align}
P(A=1) &= 0.3 \nonumber \\
P(C=1) &= 0.7 \nonumber \\
P(B=1|A,C) &= \begin{cases} 0.3\,\mathrm{if}\,A=0,C=0\\
                          0.7\,\mathrm{if}\,A=1,C=0\\
                          0.1\,\mathrm{if}\,A=0,C=1\\
                          0.9\,\mathrm{if}\,A=1,C=1 \end{cases} \nonumber \\
P(D=1|C) &= \begin{cases}   0.4\,\mathrm{if}\,C=0\\
                          0.2\,\mathrm{if}\,C=1 \end{cases} \nonumber \\
P(E=1|D) &= \begin{cases}   0.1\,\mathrm{if}\,D=0\\
                          0.1\,\mathrm{if}\,D=1 \end{cases}. \nonumber
\end{align}
Compute the probability $P(A=1|E=1,C=1)$.  Be sure to think carefully about conditional independence before pulling out your calculator.

![title](graphs/Graph2.png)

P(A=1|E=1,C=1) = P(A=1|C=1) = P(A=1,C=1)/P(C=1) = P(A=1)P(C=1)/P(C=1) = P(A=1) = 0.3
                         

## 2.Gene Sequence Clustering

### 2a. Training a Markov model (40 pts)
Load the file genes\_training.p, which is available in this homework archive.  genes\_training.p contains 2000 sequences, with each sequence $\mathbf{s}$ consisting of 20 nucleobases $s_i \in \mathrm{Nu},\; \mathrm{Nu} = \{A,T,G,C\}$.  Each of these sequences is generated from one of two separate Markov processes.  The label (aka class) of the process that generated the sequence is given in the dataset. 

Learn the Markov model for each class given the training data.  **To do this, for each class compute the prior probability $\mathbf{\pi}_c$ of each nucleobase (the relative frequency of each nucleobase for each class, a vector of length 4) and the matrix of transition probabilities** 
$$
\mathcal{A}_{c,kj} = P(s_{i+1}=\mathrm{Nu}_j|s_{i}=\mathrm{Nu}_k),
$$ 
which is a 4 by 4 matrix.  As a quick sanity check, each row of $\mathcal{A}_c$ should sum to one.  **Using these priors and transition matrices, write a function that generates a new sequence given the class**, i.e. simulates a data point. 

In [5]:
import pickle
import numpy as np

# Define some useful constants
N_nucleobases = 4
N_classes = 2
nucleobases = ['A','T','G','C']

# Load the training data using pickle
sequences,labels = pickle.load(open('genes_training.p','rb'))

# Initialize the class priors and transition matrices
pi_0 = np.zeros((N_nucleobases))
pi_1 = np.zeros((N_nucleobases))

A_0 = np.zeros((N_nucleobases,N_nucleobases))
A_1 = np.zeros((N_nucleobases,N_nucleobases))

##### Train prior #####

# Loop over all of the sequences and labels
for s,l in zip(sequences,labels):
    sequence_length = len(s)
    for p in range(sequence_length):
        if l == 0:
            pi_0[nucleobases.index(s[p])] += 1
        else:
            pi_1[nucleobases.index(s[p])] += 1
    

# Convert from counts to probabilities by normalizing
pi_0/=pi_0.sum()
pi_1/=pi_1.sum()

##### Train transition matrix #####
for s,l in zip(sequences,labels):
    sequence_length = len(s)
    for p in range(sequence_length-1):
        if l == 0:
            A_0[nucleobases.index(s[p])][nucleobases.index(s[p+1])] += 1
        else:
            A_1[nucleobases.index(s[p])][nucleobases.index(s[p+1])] += 1
        
    
# Convert from counts to probabilities by row normalization
A_0/=A_0.sum(axis=1)[:,np.newaxis]
A_1/=A_1.sum(axis=1)[:,np.newaxis]

##### Generate a synthetic sequence #####
def generate_new_sequence(A,pi,n=20):
    """  
    Arguments:
    A -> Nucleobase transition matrix
    pi -> Prior
    n -> length of sequence to generate
    """
    # Draw from the prior for the first nucleobase
    s = [np.random.choice(nucleobases,p=pi)]
    s.extend(np.random.choice(nucleobases,19,p=A[nucleobases.index(s[len(s)-1])]))
    
    return s

print(generate_new_sequence(A_0,pi_0))


['G', 'G', 'G', 'A', 'T', 'A', 'T', 'A', 'A', 'A', 'G', 'G', 'A', 'T', 'A', 'T', 'G', 'G', 'A', 'G']


### 2b. A Markov classifier (*) (20 pts)


Having the prior and transition probabilities gives you the ability to evaluate the likelihood of a sequence for a given class as:
$$
P(\mathbf{s}) = P(s_1|\mathbf{\pi}_c) \prod_{i=1}^{n-1} P(s_{i+1}|s_{i},\mathcal{A}_c),
$$
where $\mathbf{\pi}_c$ is the class-conditioned prior probability, and $\mathcal{A}_c$ is the class-conditioned transition matrix.  Comparing the computed likelihood for a given sequence between different classes forms the basis of a classifier in a very similar manner to naive Bayes.  The difference this time is that now each random variable depends on the one before it in the sequence, whereas in naive Bayes we assumed that all the random variables were independent.    

Load the file genes\_test.p, which is similar to genes\_training.p.  **For each sequence, compute the likelihood for both classes and assign a label.  Compare this predicted label to the known one, and report the test set accuracy**.  As a point of comparison, my implementation achieved 98.7\% accuracy.  

In [6]:
sequences_test,labels_test = pickle.load(open('genes_test.p','rb'))
sequence_probabilities_0 = []
sequence_probabilities_1 = []
correct = 0
total = 0

for s,l in zip(sequences_test,labels_test):
    sequence_probabilities_0.append(pi_0[nucleobases.index(s[0])])
    sequence_probabilities_1.append(pi_1[nucleobases.index(s[0])])
    for p in range(len(s)-1):
        sequence_probabilities_0[len(sequence_probabilities_0)-1] *= A_0[nucleobases.index(s[p])][nucleobases.index(s[p+1])]
        sequence_probabilities_1[len(sequence_probabilities_1)-1] *= A_1[nucleobases.index(s[p])][nucleobases.index(s[p+1])]
    
    if sequence_probabilities_0[len(sequence_probabilities_0)-1] > sequence_probabilities_1[len(sequence_probabilities_1)-1]:
        label = 0
    else:
        label = 1
    
    if label == l:
        correct += 1
    total += 1
    
print("Accuracy = " + str(correct/total))
    


Accuracy = 0.986
